In [1]:
import numpy as np
import pandas as pd
import os
import statsmodels as sm
from sklearn.mixture import GaussianMixture
from matplotlib import pyplot as plt
from pmdarima.arima import auto_arima
import cvxpy as cp
import time
import datetime as dt
import matplotlib.dates as mdates
import yfinance as yf
import mosek

In [2]:
def yearly_sharpe(nav):
    #return/volatility
    return_yearly = nav[-1]/nav[0] - 1
    returns = (nav[1:]-nav[:-1])/nav[:-1]
    sigma_daily = np.std(returns)
    sigma_yearly = sigma_daily * np.sqrt(252)
    sharpe = return_yearly/sigma_yearly
    return sharpe

def max_drawdown(X):
    mdd = 0
    peak = X[0]
    for x in X:
        if x > peak: 
            peak = x
        dd = (peak - x) / peak
        if dd > mdd:
            mdd = dd
    return mdd

def yearly_calmar(nav):
    #return/mdd
    return_yearly = nav[-1]/nav[0] - 1
    mdd = max_drawdown(nav)
    calmar = return_yearly/mdd
    return calmar
    

In [3]:
#analysis on sp500
all_files = os.listdir(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_backtest_ew')
total_count = len(all_files)
nav_op_count = 0
nav_op_list = []
strat_ret_list = []
ew_ret_list = []
op_list = []
sharpe_op_list = []
calmar_op_list = []

for file in all_files:
    df = pd.read_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_backtest_ew\{}'.format(file))
    dates = df.Dates
    nav = df.NAV.values[:]
    ew_nav = df['E.W. NAV'].values[:]
    if nav[-1] >= ew_nav[-1]:
        nav_op_list.append(file)
        strat_ret = df['Yearly Ret'][0]
        ew_ret = df['E.W. Yearly Ret'][0]
        strat_ret_list.append(strat_ret)
        ew_ret_list.append(ew_ret)
        op_list.append((strat_ret-ew_ret)/ew_ret)
        nav_op_count += 1
        years = list(set([dt.datetime.strptime(d, "%Y-%m-%d").year for d in dates]))
        strat_sharpe_list = []
        ew_sharpe_list = []
        strat_calmar_list = []
        ew_calmar_list = []
        for year in years:
            nav_year = np.array([nav[i] for i in range(len(nav)) if dt.datetime.strptime(dates[i], "%Y-%m-%d").year == year])
            ew_nav_year = np.array([ew_nav[i] for i in range(len(nav)) if dt.datetime.strptime(dates[i], "%Y-%m-%d").year == year])
            strat_sharpe, ew_sharpe = yearly_sharpe(nav_year), yearly_sharpe(ew_nav_year)
            strat_sharpe_list.append(strat_sharpe)
            ew_sharpe_list.append(ew_sharpe)
            strat_calmar, ew_calmar = yearly_calmar(nav_year), yearly_calmar(ew_nav_year)
            strat_calmar_list.append(strat_calmar)
            ew_calmar_list.append(ew_calmar)
        yearly_comparison_df = pd.DataFrame({'Years': years, 'Strategy Sharpe': strat_sharpe_list, 'E.W. Sharpe': ew_sharpe_list,
                                            'Strategy Calmar': strat_calmar_list, 'E.W. Calmar': ew_calmar_list})
        yearly_comparison_df.to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_final_result\{}'.format(file[3:]), index=False)
        sharpe_op_list.append(sum(np.array(strat_sharpe_list) > np.array(ew_sharpe_list))/len(years))
        calmar_op_list.append(sum(np.array(strat_calmar_list) > np.array(ew_calmar_list))/len(years))
        
outperform_df = pd.DataFrame({'Outperform Configs':nav_op_list, 'Strategy Returns': strat_ret_list, 
                            'E.W. Returns': ew_ret_list, 'Return Outperformance': op_list, 
                              'Sharpe Outperform Ratio': sharpe_op_list, 'Calmar Outperform Ratio': calmar_op_list})
outperform_df.to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_final_result\summary.csv', index=False)

print('{} has outperformed'.format(nav_op_count/total_count))


0.3333333333333333 has outperformed


In [4]:
#sp500 crisis analysis

#first defining timeline of crises
dotcom_start = dt.datetime(2000, 3, 10)
dotcom_end = dt.datetime(2002, 10, 4)
dotcom_dates = [dotcom_start + dt.timedelta(days=x) for x in range(0, (dotcom_end-dotcom_start).days)]

spm_start = dt.datetime(2007, 8, 1)
spm_end = dt.datetime(2009, 7,1)
spm_dates = [spm_start + dt.timedelta(days=x) for x in range(0, (spm_end-spm_start).days)]

covid_start = dt.datetime(2020, 2, 12)
covid_end = dt.datetime(2020, 8, 18)
covid_dates = [covid_start + dt.timedelta(days=x) for x in range(0, (covid_end-covid_start).days)]

crisis_dates_dict = {'Dotcom': dotcom_dates, 'SPM': spm_dates, 'Covid': covid_dates}

def sharpe_calmar_comp(crisis):
    sharpe_op_config = []
    strat_sharpe_list = []
    ew_sharpe_list = []
    sharpe_op_list = []
    
    calmar_op_config = []
    strat_calmar_list = []
    ew_calmar_list = []
    calmar_op_list = []
    
    crisis_dates = crisis_dates_dict[crisis]
    for file in all_files:
        df = pd.read_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_backtest_ew\{}'.format(file))
        nav = df.NAV.values[:]
        ew_nav = df['E.W. NAV'].values[:]
        dates = [dt.datetime.strptime(d, '%Y-%m-%d') for d in df.Dates.values[:]]
        ind = [i for i in range(len(dates)) if dates[i] in crisis_dates]
        crisis_nav = np.take(nav, ind)
        crisis_ew_nav = np.take(ew_nav, ind)
        crisis_strat_sharpe, crisis_ew_sharpe = yearly_sharpe(crisis_nav), yearly_sharpe(crisis_ew_nav)
        crisis_strat_calmar, crisis_ew_calmar = yearly_calmar(crisis_nav), yearly_calmar(crisis_ew_nav)
        if crisis_strat_sharpe > crisis_ew_sharpe:
            sharpe_op_config.append(file[3:-4])
            strat_sharpe_list.append(crisis_strat_sharpe)
            ew_sharpe_list.append(crisis_ew_sharpe)
            sharpe_op_list.append(crisis_strat_sharpe-crisis_ew_sharpe)
        if crisis_strat_calmar > crisis_ew_calmar:            
            calmar_op_config.append(file[3:-4])
            strat_calmar_list.append(crisis_strat_calmar)
            ew_calmar_list.append(crisis_ew_calmar)
            calmar_op_list.append(crisis_strat_calmar-crisis_ew_calmar)
        sharpe_df = pd.DataFrame({'Configs': sharpe_op_config, 'Strategy Sharpe': strat_sharpe_list, 
                                 'E.W. Sharpe': ew_sharpe_list, 'Outperformance': sharpe_op_list})
        sharpe_df.to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_final_result\sp500_{}_sharpe.csv'.format(crisis),
                         index=False)
        calmar_df = pd.DataFrame({'Configs': calmar_op_config, 'Strategy Calmar': strat_calmar_list,
                                'E.W. Calmar': ew_calmar_list, 'Outperformance': calmar_op_list})
        calmar_df.to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\sp500_final_result\sp500_{}_calmar.csv'.format(crisis),
                         index=False)
        
sharpe_calmar_comp('Dotcom')
sharpe_calmar_comp('SPM')
sharpe_calmar_comp('Covid')


In [5]:
#analysis on ftse350
all_files = os.listdir(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\ftse350_backtest_ew')
total_count = len(all_files)
nav_op_count = 0
nav_op_list = []
strat_ret_list = []
ew_ret_list = []
op_list = []
sharpe_op_list = []
calmar_op_list = []

for file in all_files:
    df = pd.read_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\ftse350_backtest_ew\{}'.format(file))
    dates = df.Dates
    nav = df.NAV.values[:]
    ew_nav = df['E.W. NAV'].values[:]
    if nav[-1] >= ew_nav[-1]:
        nav_op_list.append(file)
        strat_ret = df['Yearly Ret'][0]
        ew_ret = df['E.W. Yearly Ret'][0]
        strat_ret_list.append(strat_ret)
        ew_ret_list.append(ew_ret)
        op_list.append((strat_ret-ew_ret)/ew_ret)
        nav_op_count += 1
        years = list(set([dt.datetime.strptime(d, "%Y-%m-%d").year for d in dates]))
        strat_sharpe_list = []
        ew_sharpe_list = []
        strat_calmar_list = []
        ew_calmar_list = []
        for year in years:
            nav_year = np.array([nav[i] for i in range(len(nav)) if dt.datetime.strptime(dates[i], "%Y-%m-%d").year == year])
            ew_nav_year = np.array([ew_nav[i] for i in range(len(nav)) if dt.datetime.strptime(dates[i], "%Y-%m-%d").year == year])
            strat_sharpe, ew_sharpe = yearly_sharpe(nav_year), yearly_sharpe(ew_nav_year)
            strat_sharpe_list.append(strat_sharpe)
            ew_sharpe_list.append(ew_sharpe)
            strat_calmar, ew_calmar = yearly_calmar(nav_year), yearly_calmar(ew_nav_year)
            strat_calmar_list.append(strat_calmar)
            ew_calmar_list.append(ew_calmar)
        yearly_comparison_df = pd.DataFrame({'Years': years, 'Strategy Sharpe': strat_sharpe_list, 'E.W. Sharpe': ew_sharpe_list,
                                            'Strategy Calmar': strat_calmar_list, 'E.W. Calmar': ew_calmar_list})
        yearly_comparison_df.to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\ftse350_final_result\{}'.format(file[3:]), index=False)
        sharpe_op_list.append(sum(np.array(strat_sharpe_list) > np.array(ew_sharpe_list))/len(years))
        calmar_op_list.append(sum(np.array(strat_calmar_list) > np.array(ew_calmar_list))/len(years))
        
outperform_df = pd.DataFrame({'Outperform Configs':nav_op_list, 'Strategy Returns': strat_ret_list, 
                            'E.W. Returns': ew_ret_list, 'Return Outperformance': op_list, 
                              'Sharpe Outperform Ratio': sharpe_op_list, 'Calmar Outperform Ratio': calmar_op_list})
outperform_df.to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\ftse350_final_result\summary.csv', index=False)
print('{} has outperformed'.format(nav_op_count/total_count))


0.7619047619047619 has outperformed


In [6]:
#ftse350 crisis analysis

#first defining timeline of crises
covid_start = dt.datetime(2020, 2, 12)
covid_end = dt.datetime(2020, 8, 18)
covid_dates = [covid_start + dt.timedelta(days=x) for x in range(0, (covid_end-covid_start).days)]

crisis_dates_dict = {'Covid': covid_dates}

def sharpe_calmar_comp(crisis):
    sharpe_op_config = []
    strat_sharpe_list = []
    ew_sharpe_list = []
    sharpe_op_list = []
    
    calmar_op_config = []
    strat_calmar_list = []
    ew_calmar_list = []
    calmar_op_list = []
    
    crisis_dates = crisis_dates_dict[crisis]
    for file in all_files:
        df = pd.read_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\ftse350_backtest_ew\{}'.format(file))
        nav = df.NAV.values[:]
        ew_nav = df['E.W. NAV'].values[:]
        dates = [dt.datetime.strptime(d, '%Y-%m-%d') for d in df.Dates.values[:]]
        ind = [i for i in range(len(dates)) if dates[i] in crisis_dates]
        crisis_nav = np.take(nav, ind)
        crisis_ew_nav = np.take(ew_nav, ind)
        crisis_strat_sharpe, crisis_ew_sharpe = yearly_sharpe(crisis_nav), yearly_sharpe(crisis_ew_nav)
        crisis_strat_calmar, crisis_ew_calmar = yearly_calmar(crisis_nav), yearly_calmar(crisis_ew_nav)
        if crisis_strat_sharpe > crisis_ew_sharpe:
            sharpe_op_config.append(file[3:-4])
            strat_sharpe_list.append(crisis_strat_sharpe)
            ew_sharpe_list.append(crisis_ew_sharpe)
            sharpe_op_list.append(crisis_strat_sharpe-crisis_ew_sharpe)
        if crisis_strat_calmar > crisis_ew_calmar:            
            calmar_op_config.append(file[3:-4])
            strat_calmar_list.append(crisis_strat_calmar)
            ew_calmar_list.append(crisis_ew_calmar)
            calmar_op_list.append(crisis_strat_calmar-crisis_ew_calmar)
        sharpe_df = pd.DataFrame({'Configs': sharpe_op_config, 'Strategy Sharpe': strat_sharpe_list, 
                                 'E.W. Sharpe': ew_sharpe_list, 'Outperformance': sharpe_op_list})
        sharpe_df.to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\ftse350_final_result\ftse350_{}_sharpe.csv'.format(crisis),
                         index=False)
        calmar_df = pd.DataFrame({'Configs': calmar_op_config, 'Strategy Calmar': strat_calmar_list,
                                'E.W. Calmar': ew_calmar_list, 'Outperformance': calmar_op_list})
        calmar_df.to_csv(r'C:\Users\xyyh\Desktop\Cornell\ORIE 5370\Project\ftse350_final_result\ftse350_{}_calmar.csv'.format(crisis),
                         index=False)

sharpe_calmar_comp('Covid')
